**Introduction**: <br>
1. Optimizing code to make it run faster is an iterative process: <br>
(1) Find the biggest bottleneck (slowest part of your code)<br>
(2) Try to eliminate it (you may not succeed but it's ok)<br>
(3) Repeat until your code is "fast enough" <br>

2. Profile your code: use realistic input and measure the run-time of each individual operation. 

3. You should set a goal time for your code and optimize only up to that goal.

## Measuring performance

skipped

### Limitations

skipped

## Improving performance

1. Once you've used profiling to identify a bottleneck, make it faster.<br><br>
2. Use these techniques: <br><br>
(1) Look for existing solutions <br><br>
(2) Do less work<br><br>
(3) Vectorize<br><br>
(4) Parallelize<br><br>
(5) Avoid copies <br><br>
(6) Byte-code compile<br><br>

## Code organization 

1. There are 2 traps that are easy to fall into when trying to make your code faster: <br>
(1) writing faster but incorrect code <br>
(2) writing code that you think is faster, but is actually no better. <br><br>

2. The process to make faster code: <br>
(1) Compare run time of different approaches that returns the correct result. Keep a record of everything you try, even the failures. You can use R Markdown, which makes it easy to intermingle code with detailed comments and notes. <br>
(2) Generate a representative test case. The case should be big enough to capture the essence of your problem but small enough that it takes only a few seconds to run. <br>
(3) Use this test case to quickly check that all variants return the same result. Use `stopifnot()` and `all.equal()` to check same results. <br>
(4) For real problems with fewer possible outputs, you may need more tests to make sure that an approach doesn't accidentally return the correct answer.<br>
(5) Use `microbenchmark` package to compare how long each variation takes to run. <br>
(i) For bigger problems, reduce `times` parameter so that it only takes a couple of seconds to run. <br>
(ii) Focus on median time, and use upper and lower quartiles to gauge the variability of the measurement. <br><br>

In example 3, mean() is slower because it makes 2 passes over the vector to be more numerically accurate.

In [5]:
# Example 1: get mean 

# method 1
mean1 <- function(x) mean(x)

# methdo 2: faster 
mean2 <- function(x) sum(x) / length(x)

In [6]:
# Exampel 2: 
# use stopifnot() to test if get correct results (same results)

# create vector 
x <- runif(100)

# test equality 
# since they're equal, it doesn't show anything
stopifnot(all.equal(mean1(x), mean2(x)))

In [10]:
# Example 3

# load package 
library(microbenchmark)

# microbenchmark
# RStuido gives more compact result 
microbenchmark(mean1(x), 
               mean2(x))

# output in RStuio is like this:
#> Unit: nanoseconds 
#>  exper     min    lq     median  uq     max     neval
#> mean1(x)   5,680  6,080  6,250   6,490  38,500  100
#> mean2(x)   763    1,010  1,220   1,380  17,300  100

## Has someone already solved the problem? 

Get help: <br>
(1) CRAN task views: https://cran.r-project.org/web/views/ <br>
(2) rseek: https://rseek.org/ <br>
(3) stackoverflow (R tag): https://stackoverflow.com/questions/tagged/r?sort=Newest&edited=true

## Do as little as possible

1. The easiest way to make a function faster is to let it do less work. For example, use a function tailored to a more specific type of input / output, or a more specific problem. <br><br>
(1) `rowSums()`, `colSums()`, `rowMeans()`, `colMeans()` are faster than apply() because they are vectorized. <br><br>
(2) `vapply()` is faster than sapply() because it pre-specifies the output type. <br><br>
(3) If you want to see if a vector contains a single value, `any(x == 10)` is faster than `10 %in% x`. It's because testing equality is simpler than testing inclusion in a set. <br><br>

2. Having this knowledge at fingertips requires knowing that alternative functions exists. You need to have a good vocabulary. <br>
(1) Go to chapter 4 for a start. <br>
(2) Regularly read R code:<br>
(i) R-help mailing list: https://www.r-project.org/mail.html <br>
(ii) Stackoverflow (R tag) <br><br>

3. Some functions coerce their inputs into a specific type. <br>
If your input is not the right type, the function has to do extra work. <br>
Instead, look for a function that works with your data as it is, or condier changing the way you store your data. <br>
Example: `apply()` on data frame: it ways turns its input into a matrix. Not only is this error prone (because data frame is more general than matrix), it's also slower. <br><br>

4. Other functions will do less work if you give them more information about the problem: <br>
(1) `read.csv()`: specify known column types with **colClasses**<br>
(2) `factor()`: specify known levels with **levels**<br>
(3) `cut()`: don't generate labels with **labels = FALSE** if you do't need them, or even better use `findInterval()` as mentioned in the "see also" section of the documentation. <br>
(4) `unlist(x, use.names = FALSE)` is faster than unlist(x).<br>
(5) `interaction()`: if you only need combinations that exist in the data, use **drop = TRUE**.<br><br>

5. Sometimes, you can make a function faster by avoiding method dispatch. Method dispatch in R can be costly. If you're calling a method in a tight loop, you can avoid some of the costs by doing the method lookup only once: <br>
(1) for S3, do this by calling `generic.class()` instead of generic().<br>
(2) For S4, do this by using `findMethod()` to find method, saving it to a variable, and calling that function.<br><br>

Example 1: mean.default() is a bit faster for small vectors.<br>
However, mean.default() will fail in surprising ways if x isn't numeric vector. <br><br>

6. Knowing you're dealing with a specific type of input can be another way to write faster code. <br><br>

Example 2: `as.data.frame()`is slow because it coerces each element into a data frame and then rbind() them together. <br>
If you have a named list with vectors of equal length, you can directly transform it into a data frame. This method is fast because it's dangerous: if you give it bad inputs, you get corrupt data frame <br><br>

7. Most base R functions are written for flexibility and functionality, not performance. Rewriting for your specific need can often yield substantial improvements. To do this, you need to read source code. 

In [20]:
# Example 1: mean.default() is a bit faster for small vectors

# create vector 
x <- runif(1e2)

# compare mean(), mean.default()
microbenchmark(mean(x), 
               mean.default(x))

#>                    median
#> mean(x)            5.25
#> mean.default(x)    1.64

In [24]:
# Exampel 2: 

# create functiuon: quickdf()
# this makes data frame quickly
quickdf <- function(l) {
    class(l) <- "data.frame"
    attr(l, "row.names") <- .set_row_names(length(l[[1]]))
    l
}

# create vector 
l <- lapply(1:26, function(i) runif(1e3))
names(l) <- letters 

# compare: quickdf() vs as.data.frame()
microbenchmark(quick_df = quickdf(l),
              as.data.frame = as.data.frame(l))

#>                 median
#> quick_df        21.1
#> as.data.frame   1,600

## Vectorize

1. **Vectorizing code** is not just about avoiding for loops, although that's often a step. <br>
Vectorizing is about taking a whole object approach to a problem, thinking about vectors, not scalars. <br><br>

2. Two key attributes of a vectorized function: <br>
(1) It makes many problems simpler. Instead of having to think about the components of a vector, you only think about entire vectors. <br>
(2) The loops in a vectorized function are written in C. Loops in C are much faster because they have much less overhead.<br><br>

3. Functions like apply(), lapply(), Vectorise() improve interface of a function, but don't fundamentally change performance. <br>
Using vectorization for performance means finding existing R functions that is implemented in C and most closely applied to your problem.<br><br>

4. Vectorized functions that apply to many common performance bottlenecks include: <br>
(1) `rowSums()`, `colSums()`, `rowMeans()`, `colMeans()`. <br>
These vectorized matrix functions will always be faster than using apply. You can sometimes use these functions to build other vectorized functions. <br>
```
rowAny <- function(x) rowSums(x) > 0
rowAll <- function(x) rowSums(x) == ncol(x)
```
(2) Vectorized subsetting can lead to big improvements in speed. <br>
Remember Lookup tables (character subsetting) in Section 3.4.1, and matching and merging by hand in Section 3.4.2. <br>
Also remember you can use subsetting assignment to replace multiple values in a single step. If x is a vector, matrix or data frame, then `x[is.na(x)] <- 0` will replace all NAs with 0. 
(3) If you are extracting / replacing values in scattered locations in a matrix / data frame, subset with an integer matrix, see Section 3.1.3. <br>
(4) If you are converting continuous values to categorical, make sure use cut() or findInterval(). <br>
(5) Be aware of vectorized functions like `cumsum()` and `diff()`. <br><br>

5. Matrix algebra is a general example of vectorization.<br> 
The loops are executed by highly tuned external libraries like BLAS. <br>
If you can figure out a way to use matrix algebra to solve your problem, you'll often get a very fast solution. <br><br>

6. Downside of vectorization: it makes it harder to predict how operations will scale. <br>
Vectorization won't solve every problem, and rather than torturing an existing algorithm into one that uses a vectorized approach, you're often better off writing your own vectorized function in C+++

In [ ]:
# Example 1

# create look up table 
# list of 26
lookup <- setNames(as.list(sample(100, 26)), letters)

# create 3 sample vectors (names) 
x1 <- "j"
x10 <- sample(letters, 10)
x100 <- sample(letters, 199m replace = TRUE)

# compare search time 
microbenchmark(lookup[x1], 
               lookup[x10], 
               lookup[x100])

#>                median
#> lookup[x1]     634
#> lookup[x10]    1,580
#> lookup[x100])  6,110

## Avoid copies

1. Whenever you use `c(), append(), cbind(), rbind(), paste()` to create a bigger project, R must first allocate space for the new object and then copy the old object to its new home. <br>
If you repeating this many times, like in a for loop, this is expensive. 

## Byte code compilation

1. R complier can increase speed of some code. <br>
From byte code compilation, you're more likely to get 5-10% improvement. All base R functions are byte code compiled by default. 

## Case study: t-test

skipped